In [ ]:
!pip install tensorflow

# Loading Dataset
The dataset is from https://www.kaggle.com/datasets/sergeychernykh/manga-photo-60k?resource=download

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [13]:
folder_path = 'archive'

# Configure the dataset loading parameters
batch_size = 32
image_size = (460, 660)  # Adjust the size according to your requirements

# Load the images from the folder
dataset_manga = tf.keras.preprocessing.image_dataset_from_directory(
    folder_path,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    validation_split=None
)

Found 31771 files belonging to 1 classes.


In [49]:

folder_path = 'archive2'

# Configure the dataset loading parameters
batch_size = 32
image_size = (460,660)  # Adjust the size according to your requirements

# Load the images from the folder
dataset_photo = tf.keras.preprocessing.image_dataset_from_directory(
    folder_path,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    validation_split=None
)


Found 31783 files belonging to 1 classes.
<_TakeDataset element_spec=(TensorSpec(shape=(None, 460, 660, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [22]:
#iterate over dataset1
for image_batch, labels_batch in dataset_photo.take(1):
    print(image_batch.shape)


(32, 460, 660, 3)
tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(32,), dtype=int32)


In [23]:
def make_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', input_shape=[460, 660, 3]))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [24]:
D = make_discriminator()
D.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 230, 330, 64)      1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 230, 330, 64)      0         
                                                                 
 dropout (Dropout)           (None, 230, 330, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 115, 165, 64)      36928     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 115, 165, 64)      0         
                                                                 
 dropout_1 (Dropout)         (None, 115, 165, 64)      0         
                                                                 
 flatten (Flatten)           (None, 1214400)           0

In [29]:
#create generator with the output size as the input size of the discriminator
def make_generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128 * 115 * 165, input_dim=100))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Reshape((115, 165, 128)))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Conv2D(3, (7, 7), activation='sigmoid', padding='same'))
    return model

In [31]:
G = make_generator()
G.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2428800)           245308800 
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 2428800)           0         
                                                                 
 reshape_3 (Reshape)         (None, 115, 165, 128)     0         
                                                                 
 conv2d_transpose_6 (Conv2D  (None, 230, 330, 128)     262272    
 Transpose)                                                      
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 230, 330, 128)     0         
                                                                 
 conv2d_transpose_7 (Conv2D  (None, 460, 660, 128)     262272    
 Transpose)                                           

In [37]:
#create the gan model
def make_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

gan = make_gan(G, D)

In [102]:
#create save_plot function to save the generated images
def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :])
    filename = 'generated_plot_e%03d.png' % (epoch + 1)
    plt.savefig(filename)
    plt.close()

#create generate_real_samples function to generate real images from a batchDataset
def generate_real_samples(dataset, n_samples):
    images = dataset.take(1)

    y = np.ones((n_samples, 1))
    return X, y


#create generate_fake_samples function to generate fake images
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

In [103]:
#create summarize_performance function
def summarize_performance(epoch, generator, discriminator, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n_samples)
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    print(epoch, acc_real, acc_fake)
    save_plot(x_fake, epoch)
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    generator.save(filename)

In [104]:
#train the model
def train(G, D, gan, dataset_photo, dataset_manga, batch_size, n_epochs=100, n_eval=10):
    half_batch = int(batch_size / 2)
    for i in range(n_epochs):
        for j in range(n_eval):
  
            X_real, y_real = generate_real_samples(dataset_manga, half_batch)
            X_fake, y_fake = generate_fake_samples(G, 100, half_batch)

            d_loss1, = D.train_on_batch(X_real, y_real) # update D weights on real images
            
            d_loss2, = D.train_on_batch(X_fake, y_fake) # update D weights on fake images


            X_gan = np.random.randn(batch_size, 100)
            y_gan = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(X_gan, y_gan)
            print('>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' % (i + 1, j + 1, n_eval, d_loss1, d_loss2, g_loss))


        # if (i + 1) % 10 == 0:
        #     summarize_performance(i, gan, dataset_photo, dataset_manga)

In [105]:


# generate_real_samples(, n_samples)

train(G, D, gan, dataset_photo, dataset_manga, batch_size, 100, 10)

TypeError: '_TakeDataset' object is not subscriptable